In [36]:
import librosa
import librosa.display
import os
import numpy as np
import matplotlib.pyplot as plt
import time
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn import preprocessing
import keras
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn.utils import shuffle
import logging
from sklearn.metrics import roc_auc_score
from keras.callbacks import Callback

In [2]:
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(
    physical_devices[0], enable=True
)


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
path = './data/preprocessed/'
datapartitions = ['ff1010bird_' + str(i+1) for i in range(10)] + ['warblrb10k_public_' + str(i+1) for i in range(10)]
preprocessors = ['_specs', '_mfcc', '_joint']
data = []
labels = []
for partition in datapartitions:
    data.append(np.load(path + partition + preprocessors[2] + '.npy'))
    labels.append(np.load(path + partition + '_labels.npy'))
    print(f'{partition} ready')
    
data = np.concatenate(data)#[:,:,:,np.newaxis]
labels = np.concatenate(labels).astype(int)
data, labels = shuffle(data, labels, random_state=0)

testpartitions = ['test_set' + str(i+1) for i in range(10)]
test_data = []
for partition in testpartitions:
    test_data.append(np.load(path + partition + preprocessors[2] + '.npy'))
    print(f'{partition} ready')
test_data = np.concatenate(test_data)


ff1010bird_1 ready
ff1010bird_2 ready
ff1010bird_3 ready
ff1010bird_4 ready
ff1010bird_5 ready
ff1010bird_6 ready
ff1010bird_7 ready
ff1010bird_8 ready
ff1010bird_9 ready
ff1010bird_10 ready
warblrb10k_public_1 ready
warblrb10k_public_2 ready
warblrb10k_public_3 ready
warblrb10k_public_4 ready
warblrb10k_public_5 ready
warblrb10k_public_6 ready
warblrb10k_public_7 ready
warblrb10k_public_8 ready
warblrb10k_public_9 ready
warblrb10k_public_10 ready


In [50]:

testpartitions = ['test_set' + str(i+1) for i in range(10)]
test_data = []
for partition in testpartitions:
    test_data.append(np.load(path + partition + preprocessors[2] + '.npy'))
    print(f'{partition} ready')
test_data = np.concatenate(test_data)

test_set1 ready
test_set2 ready
test_set3 ready
test_set4 ready
test_set5 ready
test_set6 ready
test_set7 ready
test_set8 ready
test_set9 ready
test_set10 ready


In [44]:
model = Sequential([
    keras.Input(shape = (32,650,6)),
    keras.layers.BatchNormalization(),
    keras.layers.Conv2D(filters = 32, kernel_size = (5,5), strides = (1,1), activation='relu',padding='same'),
    keras.layers.Conv2D(filters = 32, kernel_size = (5,5), strides = (1,1), activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.SpatialDropout2D(0.1),
    keras.layers.MaxPooling2D(pool_size=(2,2)),

    keras.layers.Conv2D(filters = 64, kernel_size = (5,5), strides = (1,1), activation='relu',padding='same'),
    keras.layers.Conv2D(filters = 64, kernel_size = (5,5), strides = (1,1), activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.SpatialDropout2D(0.1),
    keras.layers.MaxPooling2D(pool_size=(2,2)),
    
    keras.layers.Conv2D(filters = 128, kernel_size = (3,3), strides = (1,1), activation='relu',padding='same'),
    keras.layers.Conv2D(filters = 128, kernel_size = (3,3), strides = (1,1), activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.SpatialDropout2D(0.1),
    keras.layers.MaxPooling2D(pool_size=(2,2)),

    keras.layers.Conv2D(filters = 256, kernel_size = (3,3), strides = (1,1), activation='relu',padding='same'),
    keras.layers.BatchNormalization(),
    keras.layers.SpatialDropout2D(0.1),
    keras.layers.GlobalAveragePooling2D(),

    keras.layers.Flatten(),
    
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    1e-4,
    decay_steps=100,
    decay_rate=0.97
)

Callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        min_delta=0.001,
        patience=10,
        mode='auto',
        restore_best_weights=True
        )
]
optimizer_adam=keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(optimizer=optimizer_adam,
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy', tf.keras.metrics.AUC()])

In [45]:
model.summary()

Model: "sequential_14"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_66 (Batc (None, 32, 650, 6)        24        
_________________________________________________________________
conv2d_89 (Conv2D)           (None, 32, 650, 32)       4832      
_________________________________________________________________
conv2d_90 (Conv2D)           (None, 32, 650, 32)       25632     
_________________________________________________________________
batch_normalization_67 (Batc (None, 32, 650, 32)       128       
_________________________________________________________________
spatial_dropout2d_55 (Spatia (None, 32, 650, 32)       0         
_________________________________________________________________
max_pooling2d_42 (MaxPooling (None, 16, 325, 32)       0         
_________________________________________________________________
conv2d_91 (Conv2D)           (None, 16, 325, 64)     

In [46]:
start_time = time.time()
model.fit(data, labels, batch_size = 128, epochs=100, verbose=True, validation_split = 0.1, callbacks=Callbacks)

print("Training time %s minutes" % ((time.time() - start_time)/60))

Epoch 1/100
111/111 [==============================] - 32s 285ms/step - loss: 0.6341 - accuracy: 0.7223 - auc_2: 0.7737 - val_loss: 0.6814 - val_accuracy: 0.7294 - val_auc_2: 0.7451
Epoch 2/100
111/111 [==============================] - 31s 280ms/step - loss: 0.6083 - accuracy: 0.7609 - auc_2: 0.8230 - val_loss: 0.6387 - val_accuracy: 0.7358 - val_auc_2: 0.7754
Epoch 3/100
111/111 [==============================] - 31s 281ms/step - loss: 0.6004 - accuracy: 0.7786 - auc_2: 0.8445 - val_loss: 0.6258 - val_accuracy: 0.7339 - val_auc_2: 0.7997
Epoch 4/100
111/111 [==============================] - 31s 281ms/step - loss: 0.5918 - accuracy: 0.8002 - auc_2: 0.8605 - val_loss: 0.6241 - val_accuracy: 0.7620 - val_auc_2: 0.8330
Epoch 5/100
111/111 [==============================] - 31s 280ms/step - loss: 0.5880 - accuracy: 0.8071 - auc_2: 0.8669 - val_loss: 0.5922 - val_accuracy: 0.8098 - val_auc_2: 0.8786
Epoch 6/100
111/111 [==============================] - 31s 280ms/step - loss: 0.5871 - acc

111/111 [==============================] - 27s 243ms/step - loss: 0.5496 - accuracy: 0.8859 - auc_2: 0.9262 - val_loss: 0.5644 - val_accuracy: 0.8628 - val_auc_2: 0.9141
Epoch 45/100
111/111 [==============================] - 27s 245ms/step - loss: 0.5473 - accuracy: 0.8919 - auc_2: 0.9277 - val_loss: 0.5658 - val_accuracy: 0.8711 - val_auc_2: 0.9162
Epoch 46/100
111/111 [==============================] - 27s 242ms/step - loss: 0.5477 - accuracy: 0.8916 - auc_2: 0.9257 - val_loss: 0.5681 - val_accuracy: 0.8698 - val_auc_2: 0.9135
Epoch 47/100
111/111 [==============================] - 27s 242ms/step - loss: 0.5472 - accuracy: 0.8922 - auc_2: 0.9283 - val_loss: 0.5643 - val_accuracy: 0.8724 - val_auc_2: 0.9160
Epoch 48/100
111/111 [==============================] - 27s 242ms/step - loss: 0.5477 - accuracy: 0.8907 - auc_2: 0.9273 - val_loss: 0.5678 - val_accuracy: 0.8762 - val_auc_2: 0.9143
Epoch 49/100
111/111 [==============================] - 27s 242ms/step - loss: 0.5479 - accuracy: 

In [47]:
model.save('./models/samuli3')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./models/samuli3\assets


In [51]:
test_data.shape

(4510, 32, 650, 6)